In [1]:
import carla
import time
import cv2
import numpy as np
import math
import sys
import os
import matplotlib.pyplot as plt
import heapq
from matplotlib.widgets import Button
import pickle

In [3]:
import carla
import time
import cv2
import numpy as np
import math
import sys
import os
import matplotlib.pyplot as plt
import heapq
from matplotlib.widgets import Button
import pickle

# Add path to CARLA Python API
sys.path.append('D:/CARLA/CARLA_0.9.15/WindowsNoEditor/PythonAPI/carla')  # Adjust path if needed
from agents.navigation.global_route_planner import GlobalRoutePlanner
from agents.navigation.local_planner import RoadOption

# Constants from the original code
PREFERRED_SPEED = 35
SPEED_THRESHOLD = 2
MAX_STEER_DEGREES = 45
STEERING_CONVERSION = 80
CAMERA_POS_Z = 3
CAMERA_POS_X = -5

# Text display parameters
font = cv2.FONT_HERSHEY_SIMPLEX
org = (30, 30)
org2 = (30, 50)
org3 = (30, 70)
org4 = (30, 90)
org5 = (30, 110)
fontScale = 0.5
color = (255, 255, 255)
thickness = 1

def maintain_speed(s):
    if s >= PREFERRED_SPEED:
        return 0
    elif s < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.7
    else:
        return 0.3

def angle_between(v1, v2):
    return math.degrees(np.arctan2(v1[1], v1[0]) - np.arctan2(v2[1], v2[0]))

def get_angle(car, wp_location):
    vehicle_pos = car.get_transform()
    car_x = vehicle_pos.location.x
    car_y = vehicle_pos.location.y
    wp_x = wp_location.x
    wp_y = wp_location.y
    
    # Calculate the difference vector
    dx = wp_x - car_x
    dy = wp_y - car_y
    
    # Check for division by zero (vehicle is at the waypoint location)
    magnitude = math.sqrt(dx*dx + dy*dy)
    if magnitude < 0.01:  # If we're very close to the waypoint (within 1cm)
        return 0.0
    
    # vector to waypoint (normalized)
    x = dx / magnitude
    y = dy / magnitude
    
    #car vector
    car_vector = vehicle_pos.get_forward_vector()
    degrees = angle_between((x,y),(car_vector.x,car_vector.y))

    return degrees

def load_waypoints(filename):
    waypoints = []
    try:
        with open(filename, 'r') as f:
            # Skip header line
            next(f)
            for line in f:
                if line.strip():
                    parts = line.strip().split(',')
                    if len(parts) >= 4:  # Need x, y, z, yaw
                        x, y, z, yaw = float(parts[0]), float(parts[1]), float(parts[2]), float(parts[3])
                        waypoints.append({
                            'location': carla.Location(x=x, y=y, z=z),
                            'yaw': yaw
                        })
        print(f"Loaded {len(waypoints)} waypoints from {filename}")
        return waypoints
    except Exception as e:
        print(f"Error loading waypoints: {e}")
        return []

def visualize_and_select_waypoints(waypoints):
    """
    Visualize all waypoints and let the user select start, end, and checkpoints.
    Returns the indices of selected waypoints.
    """
    import matplotlib
    matplotlib.use('TkAgg')  # Use TkAgg backend for better interactivity
    
    # Extract x, y coordinates from waypoints for plotting
    x_coords = [wp['location'].x for wp in waypoints]
    y_coords = [wp['location'].y for wp in waypoints]
    
    # Create a plot
    plt.figure(figsize=(14, 10))
    ax = plt.gca()
    
    # Plot all waypoints
    plt.scatter(x_coords, y_coords, s=1, c='blue', alpha=0.5, label='All Waypoints')
    plt.axis('equal')
    plt.title('Select Waypoints: Click on map to choose points')
    plt.xlabel('X coordinate')
    plt.ylabel('Y coordinate')
    plt.grid(True)
    
    # Store selected waypoints
    selected_points = []
    selected_indices = []
    start_point = None
    end_point = None
    
    # Create an axis for the buttons
    plt.subplots_adjust(bottom=0.2)
    start_ax = plt.axes([0.2, 0.05, 0.15, 0.075])
    checkpoint_ax = plt.axes([0.4, 0.05, 0.15, 0.075])
    end_ax = plt.axes([0.6, 0.05, 0.15, 0.075])
    done_ax = plt.axes([0.8, 0.05, 0.15, 0.075])
    
    # Selection state
    selection_mode = ["start"]  # Use a list for mutable reference
    
    # Text annotation for instructions
    status_text = plt.figtext(0.5, 0.15, "First click 'Start Point' button, then click on the map", 
                            ha="center", fontsize=12, bbox={"facecolor":"yellow", "alpha":0.8})
    
    def update_status(text):
        status_text.set_text(text)
        plt.draw()
    
    def on_click(event):
        nonlocal start_point, end_point
        
        # Check if click is within the plotting area
        if event.inaxes != ax:
            return
        
        print(f"Map clicked at coordinates: ({event.xdata:.2f}, {event.ydata:.2f})")
        
        # Different actions based on selection mode
        if selection_mode[0] == "start":
            # Clear previous start if exists
            if start_point:
                start_point.remove()
            
            # Find closest waypoint
            closest_idx = find_closest_waypoint(waypoints, event.xdata, event.ydata)
            wp = waypoints[closest_idx]
            
            # Plot the selected point
            start_point = ax.scatter(wp['location'].x, wp['location'].y, s=100, c='green', marker='o', zorder=5)
            
            # Add label
            plt.annotate('START', (wp['location'].x, wp['location'].y), 
                        color='green', fontsize=12, ha='center', va='bottom', zorder=5)
            
            # Update selected indices (ensure it's the first in the list)
            if closest_idx in selected_indices:
                selected_indices.remove(closest_idx)
            selected_indices.insert(0, closest_idx)
            
            plt.draw()
            update_status(f"Start point selected at ({wp['location'].x:.2f}, {wp['location'].y:.2f}). Now add checkpoints or end point.")
            
        elif selection_mode[0] == "checkpoint":
            # Find closest waypoint
            closest_idx = find_closest_waypoint(waypoints, event.xdata, event.ydata)
            wp = waypoints[closest_idx]
            
            # If this waypoint is already selected, skip
            if closest_idx in selected_indices:
                update_status(f"This waypoint is already selected. Pick a different one.")
                return
                
            # Plot the selected checkpoint
            checkpoint = ax.scatter(wp['location'].x, wp['location'].y, s=80, c='orange', marker='x', zorder=5)
            
            # Add label
            cp_num = len([i for i in selected_indices if i != selected_indices[0] and (len(selected_indices) < 2 or i != selected_indices[-1])]) + 1
            plt.annotate(f'CP {cp_num}', (wp['location'].x, wp['location'].y), 
                        color='orange', fontsize=10, ha='center', va='bottom', zorder=5)
            
            selected_points.append(checkpoint)
            
            # Add to selected indices (keep order: start, checkpoints, end)
            if len(selected_indices) >= 2:  # If we have start and end
                selected_indices.insert(-1, closest_idx)  # Insert before end
            else:
                selected_indices.append(closest_idx)  # Just append
                
            plt.draw()
            update_status(f"Checkpoint {cp_num} added at ({wp['location'].x:.2f}, {wp['location'].y:.2f}). Add more checkpoints or select end point.")
            
        elif selection_mode[0] == "end":
            # Clear previous end if exists
            if end_point:
                end_point.remove()
                
            # Find closest waypoint
            closest_idx = find_closest_waypoint(waypoints, event.xdata, event.ydata)
            wp = waypoints[closest_idx]
            
            # Plot the selected point
            end_point = ax.scatter(wp['location'].x, wp['location'].y, s=100, c='red', marker='o', zorder=5)
            
            # Add label
            plt.annotate('END', (wp['location'].x, wp['location'].y), 
                        color='red', fontsize=12, ha='center', va='bottom', zorder=5)
            
            # Update selected indices (ensure it's the last in the list)
            if closest_idx in selected_indices:
                selected_indices.remove(closest_idx)
            selected_indices.append(closest_idx)
            
            plt.draw()
            update_status(f"End point selected at ({wp['location'].x:.2f}, {wp['location'].y:.2f}). Click 'Done' when finished.")
    
    # Button click handlers
    def on_start_click(event):
        selection_mode[0] = "start"
        update_status("Click on the map to select a START point")
        print("Mode: Select start point")
        
    def on_checkpoint_click(event):
        selection_mode[0] = "checkpoint"
        update_status("Click on the map to add CHECKPOINT points (can add multiple)")
        print("Mode: Add checkpoints")
        
    def on_end_click(event):
        selection_mode[0] = "end"
        update_status("Click on the map to select an END point")
        print("Mode: Select end point")
        
    def on_done_click(event):
        if len(selected_indices) < 2:
            update_status("ERROR: You must select at least a start and end point before clicking Done!")
        else:
            plt.close()
    
    # Create buttons with distinct colors
    start_button = Button(start_ax, 'Start Point', color='lightgreen', hovercolor='green')
    start_button.on_clicked(on_start_click)
    
    checkpoint_button = Button(checkpoint_ax, 'Add Checkpoint', color='lightsalmon', hovercolor='orange')
    checkpoint_button.on_clicked(on_checkpoint_click)
    
    end_button = Button(end_ax, 'End Point', color='lightpink', hovercolor='red')
    end_button.on_clicked(on_end_click)
    
    done_button = Button(done_ax, 'Done', color='lightblue', hovercolor='blue')
    done_button.on_clicked(on_done_click)
    
    # Connect the click event and key event
    cid = plt.gcf().canvas.mpl_connect('button_press_event', on_click)
    
    # Show plot in interactive mode
    plt.show()
    
    # Ensure we have at least a start and end point
    if len(selected_indices) < 2:
        print("Warning: You need to select at least a start and end point.")
        return []
    
    print(f"Selected waypoints: {selected_indices}")
    return selected_indices

def find_closest_waypoint(waypoints, x, y):
    """Find the index of the waypoint closest to (x, y)"""
    min_dist = float('inf')
    closest_idx = -1
    
    for i, wp in enumerate(waypoints):
        dist = ((wp['location'].x - x)**2 + (wp['location'].y - y)**2)**0.5
        if dist < min_dist:
            min_dist = dist
            closest_idx = i
            
    return closest_idx

def a_star_pathfinding(waypoints, start_idx, end_idx, checkpoint_indices=None):
    """
    Implements A* algorithm to find the shortest path from start to end,
    passing through all checkpoints if provided.
    """
    if checkpoint_indices is None:
        checkpoint_indices = []
    
    # Add start and end to checkpoints for processing
    all_points = [start_idx] + checkpoint_indices + [end_idx]
    final_path = []
    
    # Find path between each consecutive pair of points
    for i in range(len(all_points) - 1):
        from_idx = all_points[i]
        to_idx = all_points[i+1]
        
        # Run A* between these two points
        segment_path = a_star_search(waypoints, from_idx, to_idx)
        
        # Add this segment to the final path (avoiding duplicates)
        if final_path and segment_path:
            final_path.extend(segment_path[1:])  # Skip the first point as it's already in the path
        else:
            final_path.extend(segment_path)
            
    return final_path

def a_star_search(waypoints, start_idx, goal_idx):
    """
    A* search algorithm to find the shortest path between two waypoints.
    """
    # Initialize open and closed sets
    open_set = [(0, start_idx)]  # Priority queue with (f_score, waypoint_idx)
    came_from = {}
    
    # g_score is cost from start to current
    g_score = {i: float('inf') for i in range(len(waypoints))}
    g_score[start_idx] = 0
    
    # f_score is estimated total cost (g_score + heuristic)
    f_score = {i: float('inf') for i in range(len(waypoints))}
    f_score[start_idx] = heuristic(waypoints, start_idx, goal_idx)
    
    # Using a set to track what's in the open list for faster lookup
    open_set_hash = {start_idx}
    
    while open_set:
        # Get the waypoint with lowest f_score
        current_f, current_idx = heapq.heappop(open_set)
        open_set_hash.remove(current_idx)
        
        # If we reached the goal, reconstruct and return the path
        if current_idx == goal_idx:
            path = []
            while current_idx in came_from:
                path.append(current_idx)
                current_idx = came_from[current_idx]
            path.append(start_idx)
            path.reverse()
            return path
        
        # Check neighbors (nearby waypoints)
        neighbors = find_neighbors(waypoints, current_idx)
        for neighbor_idx in neighbors:
            # Calculate tentative g_score
            tentative_g_score = g_score[current_idx] + calculate_distance(waypoints, current_idx, neighbor_idx)
            
            if tentative_g_score < g_score[neighbor_idx]:
                # This path is better, record it
                came_from[neighbor_idx] = current_idx
                g_score[neighbor_idx] = tentative_g_score
                f_score[neighbor_idx] = tentative_g_score + heuristic(waypoints, neighbor_idx, goal_idx)
                
                if neighbor_idx not in open_set_hash:
                    heapq.heappush(open_set, (f_score[neighbor_idx], neighbor_idx))
                    open_set_hash.add(neighbor_idx)
    
    # No path found
    return []

def heuristic(waypoints, current_idx, goal_idx):
    """Calculate heuristic (Euclidean distance)"""
    current = waypoints[current_idx]['location']
    goal = waypoints[goal_idx]['location']
    return ((current.x - goal.x)**2 + (current.y - goal.y)**2)**0.5

def calculate_distance(waypoints, idx1, idx2):
    """Calculate the distance between two waypoints"""
    wp1 = waypoints[idx1]['location']
    wp2 = waypoints[idx2]['location']
    return ((wp1.x - wp2.x)**2 + (wp1.y - wp2.y)**2)**0.5

def find_neighbors(waypoints, idx, max_neighbors=20, max_distance=5.0):
    """Find neighbors for a waypoint (closest waypoints within max_distance)"""
    distances = []
    current = waypoints[idx]['location']
    
    for i, wp in enumerate(waypoints):
        if i == idx:
            continue
        dist = ((current.x - wp['location'].x)**2 + (current.y - wp['location'].y)**2)**0.5
        if dist <= max_distance:
            distances.append((dist, i))
    
    # Sort by distance and take the closest max_neighbors
    distances.sort()
    return [i for _, i in distances[:max_neighbors]]

def visualize_path(waypoints, path_indices):
    """Visualize the calculated path"""
    import matplotlib
    matplotlib.use('TkAgg')  # Use TkAgg backend for better interactivity
    
    # Extract x, y coordinates from all waypoints
    x_coords = [wp['location'].x for wp in waypoints]
    y_coords = [wp['location'].y for wp in waypoints]
    
    # Extract path waypoints
    path_x = [waypoints[idx]['location'].x for idx in path_indices]
    path_y = [waypoints[idx]['location'].y for idx in path_indices]
    
    # Create a plot
    plt.figure(figsize=(14, 10))
    ax = plt.gca()
    
    # Plot all waypoints and the path
    plt.scatter(x_coords, y_coords, s=1, c='blue', alpha=0.3, label='All Waypoints')
    plt.plot(path_x, path_y, 'r-', linewidth=3, label='Calculated Path')
    
    # Calculate path length
    path_length = 0
    for i in range(len(path_indices)-1):
        idx1 = path_indices[i]
        idx2 = path_indices[i+1]
        wp1 = waypoints[idx1]['location']
        wp2 = waypoints[idx2]['location']
        segment_length = ((wp1.x - wp2.x)**2 + (wp1.y - wp2.y)**2)**0.5
        path_length += segment_length
    
    # Highlight the start and end points
    plt.scatter(path_x[0], path_y[0], s=150, c='green', marker='o', label='Start', zorder=5)
    plt.annotate('START', (path_x[0], path_y[0]), 
                color='green', fontsize=12, ha='center', va='bottom', zorder=5)
    
    plt.scatter(path_x[-1], path_y[-1], s=150, c='red', marker='o', label='End', zorder=5)
    plt.annotate('END', (path_x[-1], path_y[-1]), 
                color='red', fontsize=12, ha='center', va='bottom', zorder=5)
    
    # Highlight checkpoints if they exist (any point that's not start or end)
    if len(path_indices) > 2:
        checkpoint_indices = [i for i in range(1, len(path_indices)-1)]
        
        for i, cp_idx in enumerate(checkpoint_indices):
            cp_x = path_x[cp_idx]
            cp_y = path_y[cp_idx]
            plt.scatter(cp_x, cp_y, s=100, c='orange', marker='x', zorder=5)
            plt.annotate(f'CP {i+1}', (cp_x, cp_y), 
                        color='orange', fontsize=10, ha='center', va='bottom', zorder=5)
    
    # Add some path statistics
    path_info = (
        f"Path Information:\n"
        f"- Total length: {path_length:.2f} meters\n"
        f"- Waypoints: {len(path_indices)}\n"
        f"- Checkpoints: {len(path_indices)-2 if len(path_indices) > 2 else 0}"
    )
    plt.figtext(0.02, 0.02, path_info, fontsize=12, 
               bbox=dict(facecolor='white', alpha=0.8, boxstyle='round,pad=0.5'))
    
    plt.axis('equal')
    plt.title('Calculated Path - Review before navigation', fontsize=16)
    plt.xlabel('X coordinate (meters)')
    plt.ylabel('Y coordinate (meters)')
    plt.grid(True)
    plt.legend(loc='upper right')
    
    # Add a status text
    status_text = plt.figtext(0.5, 0.15, "Review the path and click 'Continue' to start navigation", 
                            ha="center", fontsize=14, bbox={"facecolor":"yellow", "alpha":0.8})
    
    # Add a button to close the window
    plt.subplots_adjust(bottom=0.2)
    close_ax = plt.axes([0.4, 0.05, 0.2, 0.075])
    close_button = Button(close_ax, 'Continue to Navigation', color='lightgreen', hovercolor='green')
    
    def on_close_click(event):
        status_text.set_text("Starting navigation...")
        plt.draw()
        plt.pause(1)  # Show "Starting navigation..." message for 1 second
        plt.close()
        
    close_button.on_clicked(on_close_click)
    
    # Show the plot
    plt.show()

def setup_world_and_vehicle(waypoints):
    """Setup the CARLA world and spawn a vehicle at the first waypoint with correct orientation."""
    # Connect to the CARLA server
    client = carla.Client('localhost', 2000)
    client.set_timeout(10.0)  # Increased timeout for loading
    
    world = client.get_world()
    
    # Get spawn points
    spawn_points = world.get_map().get_spawn_points()
    
    # Look for a blueprint of Mini car
    vehicle_bp = world.get_blueprint_library().filter('*mini*')
    
    # If we have waypoints, find the closest spawn point to the first waypoint
    if waypoints:
        first_waypoint = waypoints[0]['location']
        
        # Find the closest spawn point to the first waypoint
        closest_spawn_idx = 0
        closest_distance = float('inf')
        
        for i, spawn_point in enumerate(spawn_points):
            distance = spawn_point.location.distance(first_waypoint)
            if distance < closest_distance:
                closest_distance = distance
                closest_spawn_idx = i
        
        start_point = spawn_points[closest_spawn_idx]
        print(f"Using spawn point {closest_spawn_idx} which is {closest_distance:.2f}m from first waypoint")
    else:
        # If no waypoints, use the first spawn point
        start_point = spawn_points[0]
    
    # Try to spawn the vehicle
    vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)
    
    # If we couldn't spawn at the chosen point, try others
    if vehicle is None:
        print("Could not spawn at preferred location, trying alternatives...")
        for sp in spawn_points:
            vehicle = world.try_spawn_actor(vehicle_bp[0], sp)
            if vehicle is not None:
                break
    
    if vehicle is None:
        raise Exception("Could not spawn vehicle!")
    
    # If we have multiple waypoints, calculate the direction to face
    if len(waypoints) > 1:
        # Calculate yaw to face the second waypoint from the first
        wp1 = waypoints[0]['location']
        wp2 = waypoints[1]['location']
        
        # Calculate direction vector from wp1 to wp2
        dx = wp2.x - wp1.x
        dy = wp2.y - wp1.y
        
        # Calculate the yaw angle (in degrees) for this direction
        # atan2 returns angle in radians, convert to degrees
        # Add 90 degrees because in CARLA 0 is north (Y-axis), but atan2 considers 0 as east (X-axis)
        calculated_yaw = math.degrees(math.atan2(dy, dx))
        
        # Fix angle to be in CARLA's coordinate system
        # In CARLA, 0 degrees points along the Y axis, and angles increase clockwise
        corrected_yaw = (0 - calculated_yaw) % 360
        
        print(f"Setting vehicle to face path direction with yaw: {corrected_yaw:.2f} degrees")
    else:
        # If only one waypoint, use its yaw
        corrected_yaw = waypoints[0]['yaw']
    
    # Create a transform with the first waypoint location and calculated yaw
    initial_transform = carla.Transform(
        waypoints[0]['location'],
        carla.Rotation(pitch=0, yaw=corrected_yaw, roll=0)
    )
    
    # Teleport the vehicle to the first waypoint with correct orientation
    vehicle.set_transform(initial_transform)
    print(f"Teleported vehicle to first waypoint: {waypoints[0]['location']}")
    
    return world, vehicle

def setup_camera(world, vehicle):
    """Set up a camera attached to the vehicle."""
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', '640')
    camera_bp.set_attribute('image_size_y', '360')
    
    camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z, x=CAMERA_POS_X))
    camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)
    
    camera_data = {'image': np.zeros((360, 640, 4))}
    
    def camera_callback(image, data_dict):
        data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    
    camera.listen(lambda image: camera_callback(image, camera_data))
    
    return camera, camera_data

def draw_route(wp, route, seconds=3.0):
    # Draw the next few points route in sim window
    if len(route)-wp < 25:  # route within 25 points from end is red
        draw_colour = carla.Color(r=255, g=0, b=0)
    else:
        draw_colour = carla.Color(r=0, g=0, b=255)
    for i in range(10):
        if wp+i < len(route)-2:
            world.debug.draw_string(route[wp+i][0].transform.location, '^', draw_shadow=False,
                color=draw_colour, life_time=seconds,
                persistent_lines=True)
    return None

def save_path(path_indices, filename="saved_path.pkl"):
    """Save the path indices to a file for future use"""
    with open(filename, 'wb') as f:
        pickle.dump(path_indices, f)
    print(f"Path saved to {filename}")

def load_path(filename="saved_path.pkl"):
    """Load previously saved path indices"""
    try:
        with open(filename, 'rb') as f:
            path_indices = pickle.load(f)
        print(f"Loaded path from {filename} with {len(path_indices)} waypoints")
        return path_indices
    except Exception as e:
        print(f"Error loading path: {e}")
        return None

def main(waypoints_file):
    """Main function with updated logic for user selection and pathfinding"""
    global world
    
    # Load waypoints
    print("Loading waypoints from file...")
    waypoints = load_waypoints(waypoints_file)
    
    if not waypoints:
        print("No waypoints loaded. Exiting.")
        return
    
    # Ask user if they want to load a saved path or create a new one
    print("\n=== Waypoint Navigation System ===")
    print("1. Load a previously saved path")
    print("2. Create a new path by selecting points")
    choice = input("Choose an option (1/2): ")
    
    if choice == '1':
        # Try to load a saved path
        print("Loading saved path...")
        path_indices = load_path()
        if not path_indices:
            print("Could not load saved path. Creating a new one instead.")
            # Enter new path creation mode
            print("\nEntering path selection mode...")
            print("- First select your starting point")
            print("- Then add optional checkpoints")
            print("- Finally select your destination")
            print("- Click the buttons in order then click on the map")
            print("\nOpening map visualization...")
            
            # Visualize waypoints and let user select points
            selected_indices = visualize_and_select_waypoints(waypoints)
            
            if len(selected_indices) < 2:
                print("At least start and end points must be selected. Exiting.")
                return
            
            # First point is start, last point is end, any in between are checkpoints
            start_idx = selected_indices[0]
            end_idx = selected_indices[-1]
            checkpoint_indices = selected_indices[1:-1] if len(selected_indices) > 2 else []
            
            # Use A* to find the path
            print("\nCalculating optimal path using A* algorithm...")
            path_indices = a_star_pathfinding(waypoints, start_idx, end_idx, checkpoint_indices)
            
            # Save the path for future use
            save_path(path_indices)
    else:
        # Enter new path creation mode
        print("\nEntering path selection mode...")
        print("- First select your starting point")
        print("- Then add optional checkpoints")
        print("- Finally select your destination")
        print("- Click the buttons in order then click on the map")
        print("\nOpening map visualization...")
        
        # Visualize waypoints and let user select points
        selected_indices = visualize_and_select_waypoints(waypoints)
        
        if len(selected_indices) < 2:
            print("At least start and end points must be selected. Exiting.")
            return
        
        # First point is start, last point is end, any in between are checkpoints
        start_idx = selected_indices[0]
        end_idx = selected_indices[-1]
        checkpoint_indices = selected_indices[1:-1] if len(selected_indices) > 2 else []
        
        # Use A* to find the path
        print("\nCalculating optimal path using A* algorithm...")
        path_indices = a_star_pathfinding(waypoints, start_idx, end_idx, checkpoint_indices)
        
        # Save the path for future use
        save_path(path_indices)
    
    if not path_indices or len(path_indices) < 2:
        print("Could not find a valid path. Exiting.")
        return
    
    # Visualize the calculated path
    print("\nDisplaying calculated path...")
    print("Review the path and click 'Continue' to start navigation.")
    visualize_path(waypoints, path_indices)
    
    # Extract the waypoints for the path
    path_waypoints = [waypoints[idx] for idx in path_indices]
    
    # Setup world and vehicle
    print("\nConnecting to CARLA simulation...")
    try:
        world, vehicle = setup_world_and_vehicle(path_waypoints)
    except Exception as e:
        print(f"Error setting up world or vehicle: {e}")
        return
    
    # Setup camera
    camera, camera_data = setup_camera(world, vehicle)
    
    # Create a route from the loaded waypoints to be compatible with draw_route
    print("Creating route visualization...")
    route = []
    for wp in path_waypoints:
        # Use the raw location data to find the nearest waypoint
        waypoint_obj = world.get_map().get_waypoint(wp['location'])
        route.append((waypoint_obj, RoadOption.LANEFOLLOW))
    
    # Wait a bit for the camera to start capturing
    print("Initializing camera...")
    time.sleep(0.5)
    
    # Create window for visualization
    cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
    cv2.imshow('RGB Camera', camera_data['image'])
    
    # Allow the vehicle to stabilize and setup camera
    time.sleep(1)
    
        # Main control loop
    quit = False
    curr_wp = 0  # Start with the first waypoint
    predicted_angle = 0
    start_time = time.time()
    destination_reached = False
    stopping_phase = False

    print(f"\n=== Starting Navigation ===")
    print(f"Following {len(path_waypoints)} waypoints")
    print(f"Press 'Q' to quit navigation at any time")

    while (curr_wp < len(path_waypoints) - 1 or stopping_phase) and not quit:
        # Carla Tick
        world.tick()

        # Visualize the upcoming waypoints if not in stopping phase
        if not stopping_phase:
            draw_route(curr_wp, route, 1)

        # Check for quit
        if cv2.waitKey(1) == ord('q'):
            quit = True
            vehicle.apply_control(carla.VehicleControl(throttle=0, steer=0, brake=1))
            print("Navigation aborted by user")
            break

        # Update image
        image = camera_data['image'].copy()

        # Calculate distance to current waypoint or final point
        current_location = vehicle.get_transform().location

        if stopping_phase:
            # When stopping, always target the final waypoint
            wp_location = path_waypoints[-1]['location']
        else:
            wp_location = path_waypoints[curr_wp]['location']

        distance = current_location.distance(wp_location)

        # Get current vehicle velocity
        v = vehicle.get_velocity()
        speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2), 0)  # km/h

        # Check if we are in stopping phase
        if not stopping_phase and curr_wp == len(path_waypoints) - 1:
            stopping_phase = True
            print("Approaching final destination, preparing to stop...")

        # Handle stopping phase
        if stopping_phase:
            # Gradual stopping based on distance and speed
            if distance < 0.5 and speed < 1:
                # We've effectively stopped
                vehicle.apply_control(carla.VehicleControl(throttle=0, steer=0, brake=1))
                if not destination_reached:
                    destination_reached = True
                    print("\n=== DESTINATION REACHED ===")
                    print(f"Final position: ({current_location.x:.2f}, {current_location.y:.2f})")
                    # Display destination reached message on screen
                    cv2.putText(image, "DESTINATION REACHED", (150, 180), cv2.FONT_HERSHEY_TRIPLEX, 
                               1.2, (0, 255, 0), 2, cv2.LINE_AA)
                    cv2.imshow('RGB Camera', image)
                    cv2.waitKey(1)

                    # Wait a moment at destination
                    time.sleep(3)
                    break

            elif distance < 5.0:
                # Close to destination, apply stronger braking based on speed
                brake_force = min(0.7, max(0.3, speed / 30.0))
                steer_input = get_angle(vehicle, wp_location) / STEERING_CONVERSION
                vehicle.apply_control(carla.VehicleControl(throttle=0, steer=steer_input, brake=brake_force))

                # Add stopping indicator to display
                cv2.putText(image, "STOPPING...", (250, 180), cv2.FONT_HERSHEY_SIMPLEX, 
                           1.0, (0, 255, 255), 2, cv2.LINE_AA)
            else:
                # Approaching destination, reduce speed
                steer_input = get_angle(vehicle, wp_location) / STEERING_CONVERSION
                throttle = 0.3 if speed < 10 else 0
                vehicle.apply_control(carla.VehicleControl(throttle=throttle, steer=steer_input))
        else:
            # Regular waypoint navigation

            # Move to next waypoint if we're close enough to the current one
            if distance < 3.0:  # Reduced from 5m to 3m for more precise following
                curr_wp += 1
                if curr_wp >= len(path_waypoints):
                    print("Reached the end of waypoints!")
                    break

            # Calculate steering angle to current waypoint
            predicted_angle = get_angle(vehicle, path_waypoints[curr_wp]['location'])

            # Process steering angle
            if predicted_angle < -300:
                predicted_angle = predicted_angle + 360
            elif predicted_angle > 300:
                predicted_angle = predicted_angle - 360

            # Limit steering to max angle
            steer_input = predicted_angle
            if predicted_angle < -MAX_STEER_DEGREES:
                steer_input = -MAX_STEER_DEGREES
            elif predicted_angle > MAX_STEER_DEGREES:
                steer_input = MAX_STEER_DEGREES

            # Convert steering to CARLA input range (-1 to +1)
            steer_input = steer_input / STEERING_CONVERSION

            # Calculate throttle based on speed
            estimated_throttle = maintain_speed(speed)

            # Apply control to vehicle
            vehicle.apply_control(carla.VehicleControl(throttle=estimated_throttle, steer=steer_input))

        # Add telemetry to image
        image = cv2.putText(image, f'Steering angle: {round(predicted_angle,3)}', org, font, fontScale, color, thickness, cv2.LINE_AA)
        image = cv2.putText(image, f'Speed: {int(speed)} km/h', org2, font, fontScale, color, thickness, cv2.LINE_AA)
        image = cv2.putText(image, f'Waypoint: {curr_wp+1}/{len(path_waypoints)}', org3, font, fontScale, color, thickness, cv2.LINE_AA)
        image = cv2.putText(image, f'Distance to target: {distance:.2f}m', org4, font, fontScale, color, thickness, cv2.LINE_AA)

        # Show elapsed time
        elapsed = time.time() - start_time
        image = cv2.putText(image, f'Time: {int(elapsed//60):02d}:{int(elapsed%60):02d}', org5, font, fontScale, color, thickness, cv2.LINE_AA)

        # Show image
        cv2.imshow('RGB Camera', image)
    
    # Final stop when we've reached the end or quit
    vehicle.apply_control(carla.VehicleControl(throttle=0, steer=0, brake=1))
    
    # Calculate total time
    total_time = time.time() - start_time
    minutes = int(total_time // 60)
    seconds = int(total_time % 60)
    
    print(f"\n=== Navigation Completed ===")
    print(f"Total time: {minutes:02d}:{seconds:02d}")
    print(f"Total waypoints traversed: {curr_wp}/{len(path_waypoints)}")
    
    # Display the final frame for a moment
    time.sleep(2)
    
    # Clean up
    print("Cleaning up simulation...")
    cv2.destroyAllWindows()
    camera.stop()
    for sensor in world.get_actors().filter('*sensor*'):
        sensor.destroy()
    vehicle.destroy()
    print("Done. Thanks for using the Waypoint Navigation System!")

if __name__ == "__main__":
    if len(sys.argv) > 1:
        waypoints_file = "D:/CARLA/CARLA_0.9.15/WindowsNoEditor/PythonAPI/examples/waypoints_main.txt"
    else:
        # Try to find the waypoints file
        waypoints_file = "D:/CARLA/CARLA_0.9.15/WindowsNoEditor/PythonAPI/examples/waypoints_main.txt"
        if not os.path.exists(waypoints_file):
            # Ask for file path
            waypoints_file = input("Enter path to waypoints file: ")
    
    if not os.path.exists(waypoints_file):
        print(f"Error: File {waypoints_file} not found!")
        sys.exit(1)
    
    main(waypoints_file)

Loading waypoints from file...
Loaded 1793 waypoints from D:/CARLA/CARLA_0.9.15/WindowsNoEditor/PythonAPI/examples/waypoints_main.txt

=== Waypoint Navigation System ===
1. Load a previously saved path
2. Create a new path by selecting points
Choose an option (1/2): 2

Entering path selection mode...
- First select your starting point
- Then add optional checkpoints
- Finally select your destination
- Click the buttons in order then click on the map

Opening map visualization...
Mode: Select start point
Map clicked at coordinates: (-64.73, 24.59)
Mode: Select end point
Map clicked at coordinates: (38.12, 40.18)
Mode: Add checkpoints
Map clicked at coordinates: (63.61, 25.79)
Map clicked at coordinates: (101.39, 123.54)
Map clicked at coordinates: (-77.32, 138.84)
Map clicked at coordinates: (-113.30, 14.70)
Map clicked at coordinates: (-0.26, -68.36)
Map clicked at coordinates: (110.39, 0.00)
Map clicked at coordinates: (0.04, 16.80)
Map clicked at coordinates: (-50.33, 112.45)
Selecte